In [1]:
import pandas as pd


In [2]:
df_price = pd.read_csv('../data/player_price.csv',',')
df_stats = pd.read_csv('../data/player_stats.csv',',')

In [3]:
df_price['Date'] = pd.to_datetime(df_price['timestamp'], unit='ms')
df_price.drop(["timestamp"], axis=1, inplace=True)
df_price['Year'], df_price['Month'] = df_price['Date'].dt.year, df_price['Date'].dt.month

In [4]:
#to rename price column for convinience and drop ps_price
prices = df_price.drop(['ps_price'],axis = 1)
prices.rename(columns={'xbox_price':'price'}, inplace = True)

In [5]:
# to make name - unique
prices['Name_ID'] = prices['player_name'] + "_" +  prices['player_id'].astype(str)

In [6]:
# separate df for players names and ids with last market price
players = prices.groupby(['player_id','Name_ID'])['price'].last().reset_index()

## Let's Plotly!

In [7]:
'''
Show some thechnics to build interactive plots with Ploty



- in case you face the error "Notebook format bla bla bla....." please do:

pip install --upgrade nbformat

- in case widjets wount work:

pip install ipywidgets --upgrade

'''

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

from ipywidgets import widgets, interact 
from IPython.display import display, clear_output

import warnings
warnings.filterwarnings("ignore")

In [8]:
init_notebook_mode(connected=True)

In [9]:
club_names_dropdown = widgets.Dropdown()
club_names_dropdown.options  = [name for name in df_stats['club'].unique()]
club_names_dropdown.value = 'Real Madrid'
club_names_dropdown.description = 'Select Club Name'

In [10]:
def select_player_names(club):
    players_in_club = players.player_id.isin(df_stats[df_stats.club == club].player_id)
    player_names_dropdown.options =  [name for name in players[players_in_club].Name_ID.sort_values()]                                      
    player_names_dropdown.value = (player_names_dropdown.options[0],) 
    player_names_dropdown.description = 'Select players name'

    
player_names_dropdown = widgets.SelectMultiple()
select_player_names('Real Madrid')

In [13]:
def view_image(player_name):
    data=[]
    
    for name in player_name:
        df = prices[prices.Name_ID == name]
        trace = go.Scatter(x=df.Date,
                           y=df.price,
                           name = name
                          )
        data.append(trace)
    layout = dict(
        title='Player price: ' + ', '.join(player_name),
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='2017 Only',
                        step='year',
                        stepmode='todate'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(thickness = 0.1),
            type='date'
        )
    )

    fig = dict(data=data, layout=layout)
    iplot(fig)


In [14]:
i = widgets.interactive(select_player_names, club = club_names_dropdown)
j = widgets.interactive(view_image, player_name = player_names_dropdown)
display(i)
display(j)
# You can select multiple items